In [1]:
# noexport

!typech efficacy_utils.ipynb

[NbConvertApp] Converting notebook efficacy_utils.ipynb to python
retention_utils.py:398: error: Name 'get_ipython' is not defined
efficacy_utils.py:40: error: Name 'get_condition_to_installs_for_random_assignment_abtest' already defined on line 27
efficacy_utils.py:159: error: Name 'get_ipython' is not defined
efficacy_utils.py:163: error: Name 'get_ipython' is not defined
efficacy_utils.py:180: error: Name 'get_ipython' is not defined
efficacy_utils.py:198: error: Name 'condition_to_mlengths' is not defined


In [2]:
from retention_utils import *

<IPython.core.display.Javascript object>

In [3]:
import jsonmemo as jsonmemo_module
jsonmemo_module.set_lowmem(True)

In [4]:
#installs_with_experiment_vars = get_installs_with_experiment_vars()

In [5]:
#print(installs_with_experiment_vars[0])

In [6]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen'
  groups = ['survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [7]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen_and_choose_difficulty_frequency'
  groups = ['survey', 'nodefault_forcedchoice_userchoice', 'survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [8]:

#for install in installs_with_experiment_vars:
@msgpackmemo1arg
def get_domain_to_intervention_to_session_lengths_for_install(install):
  seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
  
  domain_to_session_to_intervention = {}
  for x in interventions_active_for_domain_and_session:
    if x.get('val') is None:
      continue
    interventions_active = json.loads(x['val'])
    if len(interventions_active) == 0:
      continue
    intervention_name = interventions_active[0]
    domain = x['key']
    session_id = x['key2']
    if domain not in domain_to_session_to_intervention:
      domain_to_session_to_intervention[domain] = {}
    domain_to_session_to_intervention[domain][session_id] = intervention_name
  
  domain_to_intervention_to_session_lengths = {}
  for x in seconds_on_domain_per_session:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_session')
      print(x)
      continue
    domain = x['key']
    session_id = x['key2']
    time_spent = x['val']
    if domain not in domain_to_session_to_intervention:
      continue
    intervention_name = domain_to_session_to_intervention[domain].get(session_id)
    if intervention_name is None:
      continue
    if domain not in domain_to_intervention_to_session_lengths:
      domain_to_intervention_to_session_lengths[domain] = {}
    if intervention_name not in domain_to_intervention_to_session_lengths[domain]:
      domain_to_intervention_to_session_lengths[domain][intervention_name] = []
    domain_to_intervention_to_session_lengths[domain][intervention_name].append(time_spent)
  return domain_to_intervention_to_session_lengths
  

In [9]:
# noexport

for x in get_collection_names():
  if 'per_day' in x and 'visits' not in x:
    print(x)
    break

edceda6041ef668213a96ba5_synced:seconds_on_domain_per_day


In [10]:
@msgpackmemo1arg
def get_epoch_to_domain_to_time_spent(install):
  seconds_on_domain_per_day = get_collection_for_install(install, 'synced:seconds_on_domain_per_day')
  output = {}
  for x in seconds_on_domain_per_day:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_day')
      print(x)
      continue
    domain = x['key']
    epoch = x['key2']
    seconds = x['val']
    if epoch not in output:
      output[epoch] = {}
    output[epoch][domain] = seconds
  return output

In [11]:
# noexport

condition_to_installs = get_condition_to_installs_for_random_assignment_abtest()


In [12]:
# noexport

for condition,installs in condition_to_installs.items():
  for install in installs:
    print(sorted(get_epoch_to_domain_to_time_spent(install).keys()))
    break

[1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1325, 1326, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336]
[1323, 1327, 1335]
[]
[1326]
[1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335]
[]


In [13]:
import math

def make_domain_to_daily_time_dataframe():
  output = []
  for condition,installs in condition_to_installs.items():
    #condition_to_lengths[condition] = []
    for install in installs:
      for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
        total_time_spent = sum(domain_to_time_spent.values())
        output.append({
          'user': install,
          'epoch': epoch,
          'time': total_time_spent,
          'logtime': math.log(total_time_spent),
          'condition': condition,
        })
#         for domain,total_time_spent in  domain_to_time_spent.items():
#           output.append({
#             'domain': domain,
#             'user': install,
#             'epoch': epoch,
#             'time': total_time_spent,
#             'logtime': math.log(total_time_spent),
#             'condition': condition,
#           })
  return to_dataframe(output)

In [14]:
df = make_domain_to_daily_time_dataframe()

In [15]:
#print(df)

In [16]:
%Rpush df

/usr/local/lib/python3.7/dist-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning:

from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.



In [17]:
%%R
library(lme4)
#library(sjPlot)
library(lmerTest)
#library(stargazer)



Error in library(lme4) : 不存在叫‘lme4’这个名字的程辑包


/usr/local/lib/python3.7/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:

Error in library(lme4) : 不存在叫‘lme4’这个名字的程辑包




In [18]:
# %%R

# df$user <- as.factor(df$user)
# #df$domain <- as.factor(df$domain)
# df$condition <- as.factor(df$condition)
# df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard"))
# df$epoch <- as.factor(df$epoch)
# df$logtime <- as.numeric(df$logtime)
# df$time <- as.numeric(df$time)
# summary(df)



In [19]:
%%R

df$user <- as.factor(df$user)
#df$domain <- as.factor(df$domain)
df$condition <- as.factor(df$condition)
df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard"))
df$epoch <- as.factor(df$epoch)
df$logtime <- as.numeric(df$logtime)
df$time <- as.numeric(df$time)
summary(df)


#results <- lmer(logtime ~ condition + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
#show(results)
#show(summary(results))
#class(results) <- "lmerMod"
#stargazer(results)

                       user          epoch           time      
 3d8f23613f3849ed266ace58: 154   1333   : 252   Min.   :    1  
 4923f5d68ea2b45c0e582fa9: 137   1326   : 247   1st Qu.: 1824  
 4d061e26d5746a4c1202a2ee:  96   1334   : 246   Median : 5704  
 11ae57ceae0127f030b9cb45:  60   1327   : 243   Mean   : 8729  
 0a02fa66d5236d68f5ad4f64:  56   1329   : 241   3rd Qu.:12074  
 44a79b99de35752f235f8c14:  54   1328   : 240   Max.   :86400  
 (Other)                 :8762   (Other):7850                  
    logtime                         condition   
 Min.   : 0.000   survey_nochoice_nothing:1379  
 1st Qu.: 7.509   survey_nochoice_easy   :1841  
 Median : 8.649   survey_nochoice_medium :1257  
 Mean   : 8.230   survey_nochoice_hard   :1716  
 3rd Qu.: 9.399   NA's                   :3126  
 Max.   :11.367                                 
                                                


In [20]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      condition_to_lengths[condition].append(total_time_spent)
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [21]:
# noexport
# bad


condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for domain,intervention_to_session_lengths in get_domain_to_intervention_to_session_lengths_for_install(install).items():
      if domain != 'www.youtube.com':
        continue
      for intervention,session_lengths in intervention_to_session_lengths.items():
        for session_length in session_lengths:
          condition_to_lengths[condition].append(session_length)


In [22]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    all_lengths_for_user = []
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      all_lengths_for_user.append(total_time_spent)
    if len(all_lengths_for_user) == 0:
      continue
    condition_to_lengths[condition].append(np.median(all_lengths_for_user))
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [23]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      if 'www.facebook.com' not in domain_to_time_spent:
        continue
      time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      condition_to_lengths[condition].append(time_on_facebook)

In [24]:
# noexport
from scipy.stats import ttest_ind

condition_to_mlengths = {}
for condition,session_lengths in condition_to_lengths.items():
  print(condition, np.mean(session_lengths))
  condition_to_mlengths[condition] = np.mean(session_lengths) / 60

#print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_nothing']))
print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_easy']))
#print(ttest_ind(condition_to_lengths['survey_nochoice_easy'], condition_to_lengths['survey_nochoice_medium']))


survey 857.5116666666667
nodefault_forcedchoice_userchoice 886.2849264705883
survey_nochoice_hard 1181.743068391867
survey_nochoice_medium 1158.2426035502958
survey_nochoice_nothing 628.2053571428571
survey_nochoice_easy 1832.9417098445597
Ttest_indResult(statistic=-3.8441432304743293, pvalue=0.00012677234604202193)


In [25]:
from plot_utils import *


In [26]:
#from plot_utils import *
plot_dict_as_bar(condition_to_mlengths, title='Total minutes spent daily on sites', xlabel='Difficulty', ylabel='Minutes spent daily on sites', remap_labels={'survey_nochoice_nothing': 'Nothing', 'survey_nochoice_easy': 'Easy', 'survey_nochoice_medium': 'Medium', 'survey_nochoice_hard': 'Hard', 'survey': 'Experience Sampling', 'nodefault_forcedchoice_userchoice': 'Onboarding Choice'})

In [27]:
# for install in installs_with_experiment_vars:
#   #seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
#   #interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
#   res = get_domain_to_intervention_to_session_lengths_for_install(install)
#   #if len(res) > 0:
#   #  print(res)
#   #  break